In [ ]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dhyanendra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/dhyanendra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
634,"Okay, it was very good...but Best Picture? Ple...",negative
270,"Long, boring, blasphemous. Never have I been s...",negative
898,"***Might not consider this having a spoiler, b...",positive
20,"""I haven't laughed this hard since granny got ...",negative
433,I haven't read the book of this and based on t...,negative


In [13]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [14]:
df = normalize_text(df)
df.head()

,review,sentiment
634,okay good but best picture please even close m...,negative
270,long boring blasphemous never glad see ending ...,negative
898,might consider spoiler rather cautious careles...,positive
20,i laughed hard since granny got caught wringer...,negative
433,read book based adaptation bother hated every ...,negative


In [15]:
df['sentiment'].value_counts()

sentiment
positive    253
negative    247
Name: count, dtype: int64

In [16]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [17]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
634,okay good but best picture please even close m...,0
270,long boring blasphemous never glad see ending ...,0
898,might consider spoiler rather cautious careles...,1
20,i laughed hard since granny got caught wringer...,0
433,read book based adaptation bother hated every ...,0


In [18]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [25]:
vectorizer = CountVectorizer(max_features=150)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/dhyanendra.manit/mlops-capstone-project.mlflow')
dagshub.init(repo_owner='dhyanendra.manit', repo_name='mlops-capstone-project', mlflow=True)


# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


ConnectError: [Errno -3] Temporary failure in name resolution

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 150)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-08 12:53:04,441 - INFO - Starting MLflow run...
2025-06-08 12:53:06,069 - INFO - Logging preprocessing parameters...
2025-06-08 12:53:07,173 - INFO - Initializing Logistic Regression model...
2025-06-08 12:53:07,173 - INFO - Fitting the model...
2025-06-08 12:53:07,192 - INFO - Model training complete.
2025-06-08 12:53:07,193 - INFO - Logging model parameters...
2025-06-08 12:53:07,550 - INFO - Making predictions...
2025-06-08 12:53:07,551 - INFO - Calculating evaluation metrics...
2025-06-08 12:53:07,560 - INFO - Logging evaluation metrics...
2025-06-08 12:53:08,964 - INFO - Saving and logging the model...
2025/06/08 12:53:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-06-08 12:53:15,257 - INFO - Model training and logging completed in 9.19 seconds.
2025-06-08 12:53:15,258 - INFO - Accuracy: 0.63
2025-06-08 12:53:15,259 - INFO - Precision: 0.

🏃 View run loud-pig-902 at: https://dagshub.com/dhyanendra.manit/mlops-capstone-project.mlflow/#/experiments/0/runs/a293763b449a427b9dc3f71bd1fe1f3e
🧪 View experiment at: https://dagshub.com/dhyanendra.manit/mlops-capstone-project.mlflow/#/experiments/0
